In [ ]:
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 2.1MB 4.0MB/s 
     |████████████████████████████████| 901kB 39.4MB/s 
     |████████████████████████████████| 3.3MB 36.7MB/s 
     |████████████████████████████████| 1.2MB 4.3MB/s 


In [ ]:
import torch
#torch.cuda.empty_cache()
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences

from transformers import XLNetModel, XLNetTokenizer, XLNetForSequenceClassification
from transformers import AdamW

import json
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)
print(device)
print(torch.cuda.get_device_name(0))

cuda
Tesla P100-PCIE-16GB


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
root_folder = "/content/drive/My Drive/CS182-Spring2020-NLP-Project/"

Mounted at /content/drive


In [ ]:
data = []
with open(root_folder + 'dataset/training_data.jsonl', 'r') as file:
    data = [json.loads(jline) for jline in file.read().splitlines()]

In [ ]:
labels = [int(review["stars"])-1 for review in data]

In [ ]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=False, remove_space=False)
data = [tokenizer.tokenize(review["text"] + " [SEP] [CLS]") for review in data]
print ("Tokenize the first sentence:")
print (data[0])


Tokenize the first sentence:
['▁total', '▁bill', '▁horrible', '▁service', '▁over', '▁8', 'gs', '▁these', '▁crook', 's', '▁actually', '▁nerve', '▁charge', '▁us', '▁69', '▁3', '▁pills', '▁', 'i', '▁checked', '▁online', '▁pills', '▁19', '▁cents', '▁each', '▁avoid', '▁hospital', '▁', 'ers', '▁costs', '▁[', 'S', 'EP', ']', '▁[', 'CL', 'S', ']']


In [ ]:
MAX_LEN = 512

In [ ]:
data = [tokenizer.convert_tokens_to_ids(x) for x in data]

In [ ]:
data = pad_sequences(data, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [ ]:
attention_masks = []

for seq in data:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

In [ ]:
data = torch.tensor(data)
labels = torch.tensor(labels)
attention_masks = torch.tensor(attention_masks)

In [ ]:
batch_size = 8

train_data = TensorDataset(data, attention_masks, labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [ ]:
# model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=5)
model = XLNetForSequenceClassification.from_pretrained("/content/drive/My Drive/XLNet_lr_76", num_labels=5)
model.cuda()

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e

In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters,
                     lr=7e-6)

In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
epochs = 4

model.train()

for _ in trange(epochs, desc="Epoch"):
  
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  for step, batch in enumerate(train_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    optimizer.zero_grad()
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    loss = outputs[0]
    logits = outputs[1]

    loss.backward()
    optimizer.step()
    
    
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

    if step % 100 == 0:
      print(step, "\t", "Train loss: {}".format(tr_loss/nb_tr_steps))
    if step % 1000 == 0:
      model.save_pretrained("/content/drive/My Drive/XLNet_lr_76")

  model.save_pretrained("/content/drive/My Drive/XLNet_lr_76")

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

0 	 Train loss: 0.3639225959777832
100 	 Train loss: 0.3834574612060396
200 	 Train loss: 0.4155336827484529
300 	 Train loss: 0.41860145924495307
400 	 Train loss: 0.4321881761760486
500 	 Train loss: 0.43872813296740404
600 	 Train loss: 0.4378790036835607
700 	 Train loss: 0.43690837102380187
800 	 Train loss: 0.44020770846885837
900 	 Train loss: 0.4367081239575584
1000 	 Train loss: 0.4324252548833172
1100 	 Train loss: 0.43128606210463594
1200 	 Train loss: 0.43025648321872645
1300 	 Train loss: 0.43021742834335197
1400 	 Train loss: 0.43186480438949637
1500 	 Train loss: 0.4326938718512684
1600 	 Train loss: 0.4342515572142333
1700 	 Train loss: 0.43561069129898855
1800 	 Train loss: 0.4386738298676995
1900 	 Train loss: 0.4383193216884782
2000 	 Train loss: 0.43959688902392735
2100 	 Train loss: 0.4386522833688778
2200 	 Train loss: 0.43867674048335
2300 	 Train loss: 0.4391129840082808
2400 	 Train loss: 0.4361605055262488
2500 	 Train loss: 0.43793770097090345
2600 	 Train lo